# pymysql

-   파이썬에서 MySQL, MariaDB DBMS와 연동하는 다양한 함수를 제공하는 모듈
-   Python [DB API 2.0](http://www.python.org/dev/peps/pep-0249) 표준을 따른다.  db 만드는 법이 여러가지 -- 표준 누가 만드냐 - 파이썬  / 구현은 알아서, 호출은 같게
-   https://github.com/PyMySQL/PyMySQL/
-   https://pymysql.readthedocs.io/en/latest/


# 설치

-   조건
    -   python version 3.6 이상
    -   mysql version 5.6 이상
-   설치
    -   `pip install pymysql`
    -   `conda install -c conda-forge pymysql`


In [1]:
%pip install pymysql

Note: you may need to restart the kernel to use updated packages.


# 기본 작성 절차

1. Database 연결
    ```python
       connection =  pymysql.connect(host:str="DBMS 서버 ip",
                                     port:int=port번호,
                                     user:str="계정명",
                                     password:str="비밀번호",
                                     db:str="연결할데이터베이스이름")
    ```
    - port 번호 기본값: 3306
    - ContextManager 타입으로 with 구문을 이용해 작성하면 close() 작업을 자동으로 처리한다.
2. Connection을 이용해 Cursor 생성
    - Cursor: 연결된 Database에 sql문을 전송하고 select결과 조회 기능을 제공하는 객체
    ```python
        cursor = connection.cursor()
    ```
    - ContextManager 타입으로 with 구문을 이용해 작성하면 close() 작업을 자동으로 처리한다.
3. Cusror를 이용해 SQL문 실행(DB Server로 전송)
    ```python
        cursor.execute("sql문")
    ```
4. Select 결과 조회
    - select 문을 실행한 경우 cursor를 의 fetch 메소드들을 이용해 select 결과를 조회한다.
    ```python
    result = cursor.fetchall()
    ```
5. 연결 닫기
    - cursor, connection 연결을 닫는다.
    - with문을 이용할 수 있다.
    ```python
    cursor.close()
    connection.close()
    ```


# 예제

## 테이블 생성


In [8]:
create_sql = """
create table customer(
  id  int  auto_increment  primary key, -- 자동증가
  name  varchar(20) not null,
  email  varchar(50) not null unique, 
  tall   double,
  birthday  date,
  created_at  datetime  not null
)
"""
# sql 문 마지막에 `;` 은 붙이지 않는다.

In [9]:
import pymysql

try:
    conn = None  #connection을 저장할 변수
    
    # 1. Database와 연결.
    conn = pymysql.connect(
        host="127.0.0.1",    # DBMS 의 ip(host) : str
        port=3306,           # DBMS의 port 번호: int
        user='dukecaboom',        # username: str
        password="12345678",    # password: str
        db="mydb"            #  연결할 Database이름: str
    )  # 연결 성공하면 연결된 DB와 관련 작업할 수있는 기능을 제공하는 Connection객체를 반환
    
    # 2. Connection을 사용해서 Cursor 객체 생성
    #    Cursor: sql 처리를 하는 기능을 제공.( sql 전송하고 처리결과를 받을 때까지를 관리)
    cursor = conn.cursor()
        
    # 3. SQL 문 전송
    cursor.execute("drop table if exists customer") # 1
    cursor.execute(create_sql) 
    
finally:
    # 4. 연결닫기(끊기)
    if conn: # conn != None
        cursor.close() # 4-1. cursor 연결 닫기
        conn.close()   # 4-2. connection 연결 닫기

## 파이썬 타입과 연결된 sql 데이터타입 
- str - 문자열타입(char, varchar, text,...)
- int - 정수(tiny int , int, ....)
- float - 실수(float, double)
- decimal.Decimal - 실수(decimal)
- datetime.date - date
- datetime.time - time
- datetime.datetime - datetime, timestamp

> ### datetime 모듈
> - 파이썬에서 날짜, 시간을 다루는 모듈
> - 날짜 type(class): date
> - 시간 type: time
> - 날짜시간 type: datetime

In [10]:
from datetime import date, time, datetime
# 실행 시점
a = datetime.now()
print(a)
print(a.year, a.month, a.day, a.hour, a.minute, a.second)

b = date.today()
print(b)

#특정 시점의 일시 
c = date(2000, 10, 2)
print(c)

d = datetime(1990, 2, 7, 10, 22, 53)
print(d)

e = time(17, 22, 33)
print(e)

2025-04-08 10:26:02.610582
2025 4 8 10 26 2
2025-04-08
2000-10-02
1990-02-07 10:22:53
17:22:33


## DML

### insert


In [11]:
sql = "insert into customer (name, email, tall, birthday, created_at) values('이순신', 'lee1@naver.com', 185.23, '2000-09-20', now())"

In [12]:
# with 문을 이용해 connection, cursor 생성: with block을 빠져 나올 때 자동으로 close() 처리한다.
#  DML(insert/update/delete) 처리 후 commit을 실행해야 영구적으로 적용된다.
with pymysql.connect(host="127.0.0.1", port=3306, user="dukecaboom", password="12345678", db="mydb") as conn: # 커넥션이 클로즈 됨 2
    with conn.cursor() as cursor: # 커서가 클로즈 되고 1
        result = cursor.execute(sql) # 반환값: 처리행수(insert/delete/update된행수, select 조회행수) # sql에서 3row affected같이 여기서 나옴
        print("처리 행수:", result)
        conn.commit() # transaction 과 관련이 있음. # 반드시 커밋하기 # 커넥션을 기준으로 
   

처리 행수: 1


### Parameterized Query -- 위에 만약 이름 '이순신' 들어가는 곳에 값을 먼저 쓰지 않고 나중에 쓰고싶을때..

-   Parameterized Query
    -   SQL 문에서 컬럼 값이 들어가는 자리에 값대신 `%s` placeholder를 사용한뒤 execute()에서 placeholder에 넣을 값을 list나 tuple로 제공한다.
    -   동일한 쿼리문을 값을 바꿔가면서 여러번 실행할 때 유용하다.


In [16]:
import pymysql

# insert할 값 입력받기
name = input("이름:")
email = input("이메일주소:")
tall = float(input("키:"))

insert_sql = "insert into customer (name, email, tall, birthday, created_at) values (%s, %s, %s, %s, now())" # 값은 now만 세팅해놓은것

with pymysql.connect(host="127.0.0.1", port=3306, user='dukecaboom', password='12345678', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(insert_sql, [name, email, tall, "2010-01-01"]) # 리스트나 튜플로 위의 %s를 넣는다. # 앞의 세개는 insert할 값으로 정하기
        conn.commit()
        print("처리 행수:", result)

이름: 진스리
이메일주소: dukecaboom@a.com
키: 160


처리 행수: 1


#### Parameterized Query를 이용해 여러개 행 insert

- executemany() 사용  --한번에 여러개 처리 insert할때 사용한다. 
    - insert할 값들을 가진 리스트를 넣어 한번에 여러 행을 insert한다.


In [17]:
from datetime import datetime, date
datas = [
    ["김인영", "abc2@a.com", 165, date(2005, 1, 12), datetime.now()],
    ["오수철", "def2@a.com", 175, date(1995, 12, 20), datetime.now()],
    ["최유명", "ghi2@a.com", 183, date(1978, 10, 28), datetime.now()],
    ["김명수", "jkl@abc.com", 177, date(2000, 2, 12), datetime.now()],
    ["이지영", "mno@abc.com", 163, date(1995, 4, 21), datetime.now()],
    ["박명수", "pqr@abc.com", 185, date(2002, 7, 5), datetime.now()],
]

In [20]:
# for date in datas:
#     cursor.execute(insert_sql, data)

In [19]:
insert_sql = "insert into customer(name, email, tall, birthday, created_at) values(%s, %s, %s, %s, %s)"
with pymysql.connect(host="127.0.0.1", port=3306, user='dukecaboom', password='12345678', db='mydb') as conn:
    with conn.cursor() as cursor:
        cnt = cursor.executemany(insert_sql, datas) # execute 반복문 돌리는 걸 executemany 로 실행 
        conn.commit()

print("insert된 총 행수:", cnt)

insert된 총 행수: 6


### update/delete

-   코딩 절차는 insert 와 동일


In [21]:
update_sql = "update customer set tall=%s where id=%s"
tall = float(input("변경할 키: "))
cust_id = int(input("변경할 고객 ID: "))

with pymysql.connect(host="127.0.0.1", port=3306, user='dukecaboom', password='12345678', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(update_sql, [tall, cust_id])
        print("처리 행수: ", result)
        conn.commit()

변경할 키:  190.4
변경할 고객 ID:  6


처리 행수:  1


In [22]:
update_sql = "update customer set email=%s, tall=%s where id=%s"
email = input("변경할 Email주소: ")
tall = float(input("변경할 키: "))
cust_id = int(input("변경할 고객 ID: "))

with pymysql.connect(host="127.0.0.1", port=3306, user='dukecaboom', password='12345678', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(update_sql, [email, tall, cust_id])
        print("처리 행수: ", result)
        conn.commit()

변경할 Email주소:  test@test.com
변경할 키:  177.23
변경할 고객 ID:  6


처리 행수:  1


In [25]:
delete_sql = "delete from customer where tall > %s"

tall = float(input("삭제기준 키:"))

with pymysql.connect(host="127.0.0.1", port=3306, user='dukecaboom', password='12345678', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(delete_sql, [tall]) # 리스트 , 튜플 다 해도 되는데 튜플은 콤마랑 괄호 까먹으니까 리스트로 하는게 편하다 
        print("처리 행수: ", result)
        conn.commit()


    # 180 이상은 삭제하겠다. 

삭제기준 키: 180


처리 행수:  0


## select (DQL - Data Query Language) 
- 조회된 행을 보는게 아니고 조회된 값을 보는게 select

-   조회결과 조회
    -   `cursor.execute("select문")` 실행 후 cursor의 결과 조회 메소드(fetch메소드)를 이용해 결과를 받는다.
      
-   fetch메소드
    -   **fetchall()**  -- cursor.fetchall 하면 다 가져온다. 
        -   조회한 모든 행을을 반환
    -   **fetchmany(size=개수)**
        -   전체 조회한 행들 중 지정한 size개수 만큼 반환.
        -   연속적으로 실행하면 다음 size개수 만큼씩 반환한다.
        -   더 이상 조회한 결과가 없으면 빈 튜플을 반환한다.
    -   **fetchone()**
        -   조회결과 중 첫번째 행만 반환
        -   주로 pk 동등 조건으로 조회한 경우 사용 - primary key


### fetchall()


In [26]:
sql = "select id, name, tall from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='dukecaboom', password='12345678', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        print("조회행수:", result)
        resultset = cursor.fetchall()

조회행수: 5


In [28]:
resultset
# tuple(개별행 - tuple(컬럼값들 )) 튜플 안에 튜플 


((2, '진스리', 160.0),
 (3, '김인영', 165.0),
 (4, '오수철', 175.0),
 (6, '김명수', 177.23),
 (7, '이지영', 163.0))

In [29]:
resultset[0][1]

'진스리'

### 조회결과를 dictionary로 반환
- pymysql.cursors.DictCursor 사용
    - Connection 생성시 또는 Cursor 생성시 지정한다.
- key: 컬럼명, value: 컬럼값

In [30]:
sql = "select id, name, tall from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='dukecaboom', password='12345678', db='mydb') as conn:
    with conn.cursor(pymysql.cursors.DictCursor) as cursor:  # 커서 반환할때 딕셔너리 커서로 바꿔서 줘~ #커넥션에 설정도 됨(기본커서 설정)
        result = cursor.execute(sql)
        print("조회행수:", result)
        resultset = cursor.fetchall()

조회행수: 5


In [31]:
resultset
# list[dictionary{key:컬럼명, value:컬럼값}]

[{'id': 2, 'name': '진스리', 'tall': 160.0},
 {'id': 3, 'name': '김인영', 'tall': 165.0},
 {'id': 4, 'name': '오수철', 'tall': 175.0},
 {'id': 6, 'name': '김명수', 'tall': 177.23},
 {'id': 7, 'name': '이지영', 'tall': 163.0}]

In [36]:
resultset[2]["name"]

TypeError: string indices must be integers, not 'str'

### fetchone()


In [32]:
sql = "select * from customer where id = %s" 
with pymysql.connect(host="127.0.0.1", port=3306, user='dukecaboom', password='12345678', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql, [7])  #(2, )
        print("조회행수:", result)
        resultset = cursor.fetchone()
        # resultset = cursor.fetchall()

조회행수: 1


In [33]:
if resultset:  # 조회결과가 없으면 None
    print(resultset)
else:
    print("조회결과가 없음.") # 튜플로 묶어서 줌 . 

(7, '이지영', 'mno@abc.com', 163.0, datetime.date(1995, 4, 21), datetime.datetime(2025, 4, 8, 10, 35, 15))


In [37]:
resultset[2]

'mno@abc.com'

In [ ]:
fetchall[0][2] 패치올은 튜플 안에 튜플이라 [0]을 써넣어줘야함. 

### fetchmany()


In [39]:
sql = "select id, name, birthday from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='dukecaboom', password='12345678', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        print("조회행수:", result)
        resultset1 = cursor.fetchmany(size=2)  # 처음 두개
        resultset2 = cursor.fetchmany(size=2)  # 다음 두개
        resultset3 = cursor.fetchmany(size=2)
        resultset4 = cursor.fetchmany(size=2)  # 개수를 나눠서 주겠다, 조회 5개인데 2222니까 빈게 나온다. 
    

조회행수: 5


In [44]:
print(resultset1)
print(resultset2)
print(resultset3)
print(resultset4) # 더이상 가져올게 없어서 빈 튜플이 반환된다. 

((2, '진스리', datetime.date(2010, 1, 1)), (3, '김인영', datetime.date(2005, 1, 12)))
((4, '오수철', datetime.date(1995, 12, 20)), (6, '김명수', datetime.date(2000, 2, 12)))
((7, '이지영', datetime.date(1995, 4, 21)),)
()


### select문을 실행한 cursor 는 iterable 타입

-   for in 문에 select query를 실행한 cursor를 사용하면 조회결과를 한 행씩 조회할 수 있다.


In [48]:
sql = "select * from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='dukecaboom', password='12345678', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)  
        print("조회행수:", result)
        for id, name, email, tall, birthday, created_at in cursor: # 커서가 iterable타입이 되어서 for in 문에 쓸 수 있다. 셀렉트 결과를 한 행씩 한 행씩 ㄱㄱ
            print(id, name, email, tall, birthday, created_at, sep=" , ")

조회행수: 5
2 , 진스리 , dukecaboom@a.com , 160.0 , 2010-01-01 , 2025-04-08 10:32:00
3 , 김인영 , abc2@a.com , 165.0 , 2005-01-12 , 2025-04-08 10:35:15
4 , 오수철 , def2@a.com , 175.0 , 1995-12-20 , 2025-04-08 10:35:15
6 , 김명수 , test@test.com , 177.23 , 2000-02-12 , 2025-04-08 10:35:15
7 , 이지영 , mno@abc.com , 163.0 , 1995-04-21 , 2025-04-08 10:35:15


In [49]:
# for 튜플 in

In [1]:
## %%writefile customer_db.py   
# 옆에 파일 생김 


##################
# SQL 실행 코드 함수화
##################

# 모듈이 될 때는 #임포트 해와야한다. 


from datetime import date
import pymysql
def select_customer_by_id(cust_id: int) -> tuple|None:  # 파라미터로 받은 커스터머 아이디를 받음 밑에에ㅔㅔㅔㅔ 
    """
    고객 id로 고객정보를 DB에서 조회해서 반환하는 함수
    Args:
    Returns:
        tuple: 조회결과
        None: 조회결과가 없을 경우
    Raises:
    """

    sql = "select * from customer where id =%s"
    with pymysql.connect(host="127.0.0.1", port=3306, user='dukecaboom', password='12345678', db='mydb') as conn:
        with conn.cursor()as cursor:
            result = cursor.execute(sql, [cust_id]) # 여기에 !!!!! 
            return cursor.fetchone()



def select_all_customer(): ## 내용이없는함수
    """
    전체 고객정보를 조회하는 함수
    select * from customer;
    """
    pass




def update_customer(cust_id:int, name:str, email:str, tall:float, birthday:date|str) -> int:

    sql = (" update customer "
           " set name = %s, email = %s, tall = %s, birthday = %s " 
           " where id = %s ")
    with pymysql.connect(host="127.0.0.1", port=3306, user='dukecaboom', password='12345678', db='mydb') as conn:
        with conn.cursor()as cursor:
            result = cursor.execute(sql, (name, email, tall, birthday, cust_id))
            conn.commit()    # 커밋을 빼먹으면 안바뀐다ㅏ. 연결해줘야함. 
            return result

    
        

In [76]:
# '3','김인영','abc2@a.com','165','2005-01-12','2025-04-08 10:35:15'

# 이름은 안바꾸고싶으면 그대로 넣는다. 
update_customer(3, "김인영", "new123@a.com", 165, '2005-01-12')

1

In [68]:
(" update customer "
           " set name = %s, email = %s, tall = %s, birthday = %s " # " 앞뒤에 공백을 넣어!!!! 그래야 안붙는다 
           " where id = %s ")


' update customer  set name = %s, email = %s, tall = %s, birthday = %s  where id = %s '

In [64]:
#"aaaaaaaaaaaaaa\
#aaaaaaaaa" # 역슬래시를 써서 가독성을 좋게한다. 한줄이지만 두줄처럼 보이게.


(
    "aaaaaaaaaaaaaa"
    "bbbbbbbbbbbbbb"
    "cccccccccccccc"
)  # 쉼표없이 문자를 괄호 안에 넣으면 그냥 한줄로 나온다 

'aaaaaaaaaaaaaabbbbbbbbbbbbbbcccccccccccccc'

In [60]:
result = select_customer_by_id(4)
print(result)

(4, '오수철', 'def2@a.com', 175.0, datetime.date(1995, 12, 20), datetime.datetime(2025, 4, 8, 10, 35, 15))


In [59]:
result = select_customer_by_id(1)
print(result)

None
